#### python 等分切割日志文件【filesplit】
碰到这样的需求，将日志文件等分切割，可按数据量，也可以按行数等分，但不能把任一行断开【不用绝对等分】，行的格式是固定的，断开后会影响解析，尝试用python自带的读写模块写逻辑，可以逐行读取按行数切割文件，也能做到按字符量的能分，但效率不是太满意，百度后，filesplit模块刚好能满足需求，效率还高了不少，期间碰到了一篇奇葩文章，羊头狗肉，安装的是filesplit模块，下面给的demo确是fsplit。

先扔块砖,逐行读取内容，避免大量占用内存，读一行写一行，用flag记录当前已写入数据量

In [1]:
import os
file_path = "../files/a_level_1.txt"
# 按数据量进行分割
def spbysize(file_path,size=2000000,to=''):
    if not os.path.exists(file_path):
        print("文件不存在")
        return
    flag = 0
    idx = 1
    file_name = os.path.basename(file_path).split(".")
    to_file_tpl = file_name[0]+"_{}_."+file_name[1]
    if to:
        to_file_tpl = to+"/"+to_file_tpl
    to_file_name = to_file_tpl.format(idx)
    with open(file_path) as file:
        for line in file:
            with open(to_file_name,mode="a") as input_file:
                input_file.write(line)
            flag += len(line)
            if flag >= size:
                flag = 0
                idx += 1
                to_file_name = to_file_tpl.format(idx)
# 按行数进行分割
def spbylinecount(file_path,len=100000,to=''):
    if not os.path.exists(file_path):
        print("文件不存在")
        return
    flag = 0
    idx = 1
    file_name = os.path.basename(file_path).split(".")
    to_file_tpl = file_name[0]+"_{}_."+file_name[1]
    if to:
        to_file_tpl = to+"/"+to_file_tpl
    to_file_name = to_file_tpl.format(idx)
    with open(file_path) as file:
        for line in file:
            with open(to_file_name,mode="a") as input_file:
                input_file.write(line)
            flag += 1
            if flag >= len:
                flag = 0
                idx += 1
                to_file_name = to_file_tpl.format(idx)

用一个13M的文件试了一下，分割的文件没问题，挺均匀，也没断行，只是时间上太惨淡，500+秒。可能是逐行的读取方式导致，如果用read方法，可能会快一些，但会出现断行。


In [9]:
import time
file_path = "../files/a_level_1.txt"
time_start = time.time()
spbylinecount(file_path)
time.time() - time_start

592.1513235569

再看下filesplit版

- 安装：pip install filesplit
- 引入: from filesplit.split import Split
- 先声明一个Split(inputfile="",outputdir="")对象,inputfile:源文件，必传，outputdir:生成文件存储目录，必传
- 分割函数【按数据量】bysize(size=2000000,newline=True) size：单个文件大小，newline：为True的话，能保持每行的完整性，不会断行，默认False
- 分割函数 【按行数】bylinecount(100000)
- 时间量在1s以内

###### 按数据量分割

In [4]:
import time
from filesplit.split import Split
time_start = time.time()
file_path = "../files/a_level_1.txt"
split = Split(inputfile=file_path,outputdir="../files/")
split.bysize(size=2000000,newline=True)
time.time() - time_start

0.33939194679260254

###### 按行数分割

In [5]:
import time
from filesplit.split import Split
time_start = time.time()
file_path = "../files/a_level_1.txt"
split = Split(inputfile=file_path,outputdir="../files/")
split.bylinecount(10000000)
time.time() - time_start


0.2572145462036133

有尝试了个超大文件【7.63G】，70秒

In [7]:
import time
from filesplit.split import Split
time_start = time.time()
file_path = "../files/lines.txt"
split = Split(inputfile=file_path,outputdir="../files/")
split.bylinecount(10000000)
time.time() - time_start

70.94820404052734